# Assignment 2 : K-Means Clustering
## Group - 48
## P.Anurag Reddy    19CS30032
## Shashank Suroju   19CS10061

# Imports

In [492]:
import numpy as np 
import pandas as pd 
import math
import random                            # To get random numbers
from matplotlib import pyplot as plt     # To draw plots
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import homogeneity_score

In [493]:
# Reading data into a pandas dataframe
data = pd.read_csv("Indian Liver Patient Dataset (ILPD).csv")
data.head()

In [494]:
# Replacing Male, Female as 1,2 respectively in "gender"
data["gender"] = [1 if x=="Male" else 2 for x in data["gender"]]
data.head()

In [495]:
# checking if there is a missing data
data.describe()

In [496]:
# Adding the missing data in alkphos with the mean of remaining data
mean = data.alkphos.mean()
data.fillna(mean, inplace = True)

In [497]:
data.describe()

In [498]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
print(X.shape)
print(Y.shape)

## Part 1
# K-means clustering algorithm

In [499]:
# Class for K-means clustering algorithm 
class KMeans():
    def __init__(self, k=2, max_iterations=100, test=False, centroids = None):
        '''Constructor for Kmeans class'''
        
        self.K = k                                   # number of clusters
        self.max_iterations = max_iterations                   # maximum iterations
        if centroids is None:
            self.centroids = []
        else:
            self.centroids = centroids
        self.clusters = [[] for _ in range(self.K)]  # initialising k empty clusters 
        self.test = test                             # variable to check if it is a testmode(testA)
    
    def create_clusters(self, centroids):
        '''function to create clusters for the data by calculating the nearest centroid to the current point'''
        
        temp_clusters = [[] for _ in range(self.K)]       # locally initialising k empty clusters
        for index, sample in enumerate(self.X):
            centroid_index = self.get_closest_centroid(sample, centroids)   # function call to find the nearest centroid
            temp_clusters[centroid_index].append(index)                         # marking the current point into the cluster of nearest centroid
        return temp_clusters 
    
    def get_closest_centroid(self, sample, centroids):
        '''returns the closest centroid for the sample point using euclidean distance'''
        
        total_distances = [self.euclidean_distance(sample, cent) for cent in centroids]
        closest_centroid_index = np.argmin(total_distances)
        return closest_centroid_index
    
    def get_centroids(self, clusters):
        '''returns new centroids from the clusters'''
        # assign mean value of clusters to centroids
        temp_centroids = np.zeros((self.K, self.n_features))
        for cluster_index, cluster in enumerate(clusters):
            mean_cluster = np.mean(self.X[cluster], axis=0)
            temp_centroids[cluster_index] = mean_cluster
        return temp_centroids
    
    def euclidean_distance(self, x1, x2):
        '''Computes the euclidean distance between two n dimensional vectors'''
        
        dist = 0
        for i in range(len(x1)):
            dist += pow((x1[i] - x2[i]), 2)
        return math.sqrt(dist)
    
    def assign_cluster_labels(self, clusters):
        '''Classifing samples as the index of their clusters'''
        
        cluster_labels = np.empty(self.n_samples) # initialising empty labels
        for cluster_index, cluster in enumerate(clusters):
            for sample_idx in cluster:
                cluster_labels[sample_idx] = cluster_index
        return cluster_labels
    
    def is_converged(self, centroids_old, centroids):
        '''Checks whether previous centroids, current centroids are the same or not'''
        
        # distances between each old and new centroids, fol all centroids
        total_distances = [self.euclidean_distance(centroids_old[itr], centroids[itr]) for itr in range(self.K)]
        return sum(total_distances) == 0
    
    def predict(self, X):
        '''Returns the labels of the data by iterating untill the centroids doesn't change'''
        self.X = X
        self.n_samples, self.n_features = self.X.shape

        # initialize
        if self.test: # checking if it is a testmode                                     
            pass
        else:         # else initialse random clusters
            random_sample_indices = np.random.choice(self.n_samples, self.K, replace=False)
            self.centroids = [self.X[idx] for idx in random_sample_indices]

        # Optimize clusters
        for _ in range(self.max_iterations):                                # loop max_iter no of times, exit if the clusters aren't changing
            self.clusters = self.create_clusters(self.centroids)       # Assigning samples to closest centroids (create clusters)
            centroids_old = self.centroids                             # Storing the old centroids 
            self.centroids = self.get_centroids(self.clusters)         # Calculate new centroids from the clusters
            
            if self.is_converged(centroids_old, self.centroids):       # checking if clusters have changed
                break

        return self.assign_cluster_labels(self.clusters)                 # Classify samples as the index of their clusters

In [500]:
no_of_clusters = len(np.unique(Y))
km = KMeans(k=no_of_clusters, max_iterations=150)
pred_y = km.predict(X)                    # labels for the input data

# Part 2

In [501]:
# Accuracy 
acc = accuracy_score(Y, pred_y)

In [502]:
# AMI
ami = adjusted_mutual_info_score(Y.flatten(), pred_y)

In [503]:
# NMI
nmi = normalized_mutual_info_score(Y.flatten(), pred_y)

In [504]:
# ARI
ari = adjusted_rand_score(Y.flatten(), pred_y)

In [505]:
# Homogenity score
hs = homogeneity_score(Y.flatten(), pred_y)

In [506]:
# Silhoutte score
ss = silhouette_score(X, pred_y)

In [507]:
# Calinski harabasz score
chs = calinski_harabasz_score(X, pred_y)

In [508]:
print("Accuracy: ", acc)
print("AMI: ", ami)
print("NMI: ", nmi)
print("ARI: ", ari)
print("Homgenity score: ", hs)
print("Silhoutte score: ", ss)
print("Calinski harabasz score: ", chs)

# Part 3
## Determinig most suitable K

## 1. Silhoutte index 

In [509]:
silhouette_coefficients = []
for k in range(2, 11):
    kmeans = KMeans(k=k, max_iterations=150)
    pred_y = kmeans.predict(X)
    score = silhouette_score(X, pred_y)
    silhouette_coefficients.append(score)
print(silhouette_coefficients)

In [510]:
plt.plot(range(2, 11), silhouette_coefficients)
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.title("Plot for Silhoutte Coefficient")
plt.show()

## 2. Calinski harabaz score

In [511]:
calinski_harabasz = []
for k in range(2, 11):
    kmeans = KMeans(k=k, max_iterations=150)
    pred_y = kmeans.predict(X)
    score = calinski_harabasz_score(X, pred_y)
    calinski_harabasz.append(score)
print(calinski_harabasz)

In [512]:
plt.plot(range(2, 11), calinski_harabasz)
plt.xlabel("Number of Clusters")
plt.ylabel("calinski_harabasz_score")
plt.title("Plot of Calinski Scores")
plt.show()

# Part 4

In [513]:
def find_centroids(centroids,X):
    '''finds the final centroids of current data by applying Kmeans'''
    km_train = KMeans(centroids = centroids)
    pred_train = km_train.predict(X)
    return km.centroids

In [514]:
def TestA(data):
    '''Evaluating the avearge NMI by choosing k random centroids(for 50 times) ,splitting data, applying K-means on training data and
        labeling the test data using the final centroids of training data, finally calculates the clustering accuracy 
         through NMI
       '''
    temp_data = data    # duplicating the data into temp_data
    nmi_metric = []     # final average NMI's of all iterations
    for i in range(50):
        data = temp_data                                    # restoring the original data
        X = data.iloc[:, :-1].values                  
        Y = data.iloc[:, -1].values.reshape(-1,1)
        n_samples, n_features = X.shape

        random_sample_idxs = np.random.choice(n_samples, 2, replace=False)
        centroids = [X[idx] for idx in random_sample_idxs]
        new_data = data.drop(random_sample_idxs)            # removing the k centroids from the data
        
        X = new_data.iloc[:, :-1].values
        Y = new_data.iloc[:, -1].values.reshape(-1,1)

        NMIS = []                                           # Storing the current 50 NMI's
        for j in range(50):
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=391) # Splitting the data[80:20]
            new_centroids = find_centroids(centroids, X_train)       # getting the final centroids of training data
            km_test = KMeans(test = True, centroids = new_centroids) # Applying Kmeans for the test data with final centroids of training data
            pred_test = km_test.predict(X_test)                      # Labeling the test data
            z = normalized_mutual_info_score(Y_test.flatten(), pred_test) # Calculating NMI for test data
            NMIS.append(z)
        Nmi = np.array(NMIS)
        print(i+1,"Average NMI for 50 iterations: ", np.average(Nmi))
        nmi_metric.append(np.average(Nmi))
    return nmi_metric

In [515]:
nmi = TestA(data)

In [ ]:
plt.plot(range(1, 51), nmi)
plt.xlabel("ith iteration")
plt.ylabel("Nmi_metric")
plt.title("Final NMI's plot")
plt.show()